In [ ]:
from sou.reasoning.run_reasoning import run_reasoning_loop
from sou.reasoning.config import ReasoningSettings

settings = ReasoningSettings(
    model_name="groq/llama3-70b-8192",
)


INFO:httpx:HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
/Users/yulin/anaconda3/envs/agentic_reasoning1/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
prompt = "What is the capital of France?"
final_sequence = run_reasoning_loop(prompt, reasoning_settings=settings)